# cjm-text-plugin-system

> Defines standardized interfaces and data structures for text processing plugins, enabling modular NLP operations like sentence splitting, tokenization, and chunking within the cjm-plugin-system ecosystem.

## Install

```bash
pip install cjm_text_plugin_system
```